# Load AutoML model and batch predict

## Overview


This tutorial demonstrates how to use the Vertex AI SDK to load a tabular forecasting models and do batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

Learn more about [Forecasting for tabular data](https://cloud.google.com/vertex-ai/docs/tabular-data/forecasting/overview).

### Objective

In this tutorial, you learn how to create an `AutoML` tabular forecasting model from a Python script, and then do a batch prediction using the Vertex AI SDK. You can alternatively create and deploy models using the `gcloud` command-line tool or online using the Cloud Console.

This tutorial uses the following Google Cloud ML services:

- `AutoML Training`
- `Vertex AI Batch Prediction`
- `Vertex AI Model` resource

The steps performed include:

- Create a `Vertex AI Dataset` resource.
- Train an `AutoML` tabular forecasting `Model` resource.
- Obtain the evaluation metrics for the `Model` resource.
- Make a batch prediction.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook. 

In [1]:
import os

In [6]:
#!pip install --upgrade google-cloud-aiplatform

### Colab only: Uncomment the following cell to restart the kernel

In [3]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## Before you begin

### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [2]:
PROJECT_ID = "tidal-plasma-387718"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Region

You can also change the `REGION` variable used by Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [3]:
REGION = "europe-west1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [4]:
!gcloud auth list

                  Credentialed Accounts
ACTIVE  ACCOUNT
        490069743207-compute@developer.gserviceaccount.com
*       hozefa.zap1@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`



**3. Colab, uncomment and run:**

In [10]:
#from google.colab import auth
#auth.authenticate_user()

**4. Service account or other**
* See how to grant Cloud Storage permissions to your service account at https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples.

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [5]:
BUCKET_URI = f"gs://vertex_ai_ackathon_staging"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [15]:
#! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Creating gs://vertex_ai_ackathon_staging/...


### Import libraries and define constants

In [6]:
import urllib

import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

## Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [7]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# Tutorial

Now you are ready to start creating your own AutoML tabular forecasting model.

#### Location of BigQuery training data.

Now set the variable `TRAINING_DATASET_BQ_PATH` to the location of the BigQuery table. 

In [ ]:
TRAINING_DATASET_BQ_PATH = (
    "bq://bigquery-public-data:iowa_liquor_sales_forecasting.2020_sales_train"
)

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `TimeSeriesDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the `Dataset` resource.

This operation may take several minutes.

In [ ]:
dataset = aiplatform.TimeSeriesDataset.create(
    display_name="iowa_liquor_sales_train",
    bq_source=[TRAINING_DATASET_BQ_PATH],
)

time_column = "date"
time_series_identifier_column = "store_name"
target_column = "sale_dollars"

print(dataset.resource_name)

In [ ]:
COLUMN_SPECS = {
    time_column: "timestamp",
    target_column: "numeric",
    "city": "categorical",
    "zip_code": "categorical",
    "county": "categorical",
}

### Create and run training job

To train an AutoML model, you perform two steps: 1) create a training job, and 2) run the job.

#### Create training job

An AutoML training job is created with the `AutoMLForecastingTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `column_transformations`: (Optional): Transformations to apply to the input columns
- `optimization_objective`: The optimization objective to minimize or maximize.
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

The instantiated object is the job for the training pipeline.

In [ ]:
MODEL_DISPLAY_NAME = "iowa-liquor-sales-forecast-model"

training_job = aiplatform.AutoMLForecastingTrainingJob(
    display_name=MODEL_DISPLAY_NAME,
    optimization_objective="minimize-rmse",
    column_specs=COLUMN_SPECS,
)

#### Run the training pipeline

Next, you start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).
- `target_column`: The name of the column to train as the label.
- `budget_milli_node_hours`: (optional) Maximum training time specified in unit of millihours (1000 = hour).
- `time_column`: Time-series column for the forecast model.
- `time_series_identifier_column`: ID column for the time-series column.

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take up to one hour.

In [ ]:
model = training_job.run(
    dataset=dataset,
    target_column=target_column,
    time_column=time_column,
    time_series_identifier_column=time_series_identifier_column,
    available_at_forecast_columns=[time_column],
    unavailable_at_forecast_columns=[target_column],
    time_series_attribute_columns=["city", "zip_code", "county"],
    forecast_horizon=30,
    context_window=30,
    data_granularity_unit="day",
    data_granularity_count=1,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name=MODEL_DISPLAY_NAME,
    predefined_split_column_name=None,
)

## Load the model

In [8]:
MODEL_ID = "5224852866912485376"

In [9]:
model = aiplatform.Model(
    model_name = MODEL_ID ,
    project= PROJECT_ID,
    location= REGION)

In [10]:
#model = aiplatform.Model(f'/projects/{PROJECT_ID}/locations/{REGION}/models/{MODEL_ID}')

## Review model evaluation scores

After your model training has finished, you can review the evaluation scores for 

In [11]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

{'name': 'projects/490069743207/locations/europe-west1/models/5224852866912485376@1/evaluations/683772150541546803', 'metricsSchemaUri': 'gs://google-cloud-aiplatform/schema/modelevaluation/forecasting_metrics_1.0.0.yaml', 'metrics': {'weightedAbsolutePercentageError': 3.048773, 'rootMeanSquaredLogError': 0.043229423, 'meanAbsolutePercentageError': 3.0718994, 'rSquared': 0.9577837, 'meanAbsoluteError': 1.893742, 'rootMeanSquaredError': 2.784496, 'rootMeanSquaredPercentageError': 4.3391757}, 'createTime': '2023-07-09T23:10:22.872111Z', 'modelExplanation': {'meanAttributions': [{'featureAttributions': {'visibility': 0.8985698790777299, 'dew': 13.488292183194847, 'solarenergy': 0.32219049476441897, 'precipcover': 0.490410952340989, 'tempmin': 0.0, 'uvindex': 0.2681974342891148, 'datetime': 0.22211163952237087, 'sunset': 0.29475768407185876, 'sunrise': 0.3000280402955555, 'sealevelpressure': 0.24215170315333776, 'humidity': 5.728647493180775, 'windgust': 0.4041374297369094, 'windspeed': 0.

## Send a batch prediction request

Send a batch prediction to your deployed model.

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method using a BigQuery source and destination, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `bigquery_source`: BigQuery URI to a table, up to 2000 characters long. For example: `bq://projectId.bqDatasetId.bqTableId`
- `bigquery_destination_prefix`: The BigQuery dataset or table for storing the batch prediction resuls.
- `instances_format`: The format for the input instances. Since a BigQuery source is used here, this should be set to `bigquery`.
- `predictions_format`: The format for the output predictions, `bigquery` is used here to output to a BigQuery table.
- `generate_explanations`: Set to `True` to generate explanations.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [13]:
batch_predict_bq_output_dataset_name = f"hackathon_vertex_ai_predictions_{MODEL_ID}"
batch_predict_bq_output_dataset_path = "{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
batch_predict_bq_output_uri_prefix = "bq://{}.{}".format(
    PROJECT_ID, batch_predict_bq_output_dataset_name
)
# Must be the same region as batch_predict_bq_input_uri
client = bigquery.Client(project=PROJECT_ID)
bq_dataset = bigquery.Dataset(batch_predict_bq_output_dataset_path)
dataset_region = REGION  # @param {type : "string"}
bq_dataset.location = dataset_region
bq_dataset = client.create_dataset(bq_dataset)
print(
    "Created bigquery dataset {} in {}".format(
        batch_predict_bq_output_dataset_path, dataset_region
    )
)

Created bigquery dataset tidal-plasma-387718.hackathon_vertex_ai_predictions_5224852866912485376 in europe-west1


For AutoML models, manual scaling can be adjusted by setting both min and max nodes i.e., `starting_replica_count` and `max_replica_count` as the same value(in this example, set to 1). The node count can be increased or decreased as required by load.
 
`batch_predict` can export predictions either to BigQuery or GCS. This example exports to BigQuery.

In [14]:
%%time
PREDICTION_DATASET_BQ_PATH = (
    "bq://tidal-plasma-387718.hackathon_vertex_ai.xnew_automl_3_hor_sel_fea_Dallas"
)

batch_prediction_job = model.batch_predict(
    job_display_name="predictions automl 3 hor sel fea Dallas",
    bigquery_source=PREDICTION_DATASET_BQ_PATH,
    instances_format="bigquery",
    bigquery_destination_prefix=batch_predict_bq_output_uri_prefix,
    predictions_format="bigquery",
    generate_explanation=True,
    sync=False,
)

print(batch_prediction_job)

Creating BatchPredictionJob
<google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7fef4a604f40> is waiting for upstream dependencies to complete.
CPU times: user 9.5 ms, sys: 373 µs, total: 9.87 ms
Wall time: 8.92 ms
BatchPredictionJob created. Resource name: projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248
To use this BatchPredictionJob in another session:
bpj = aiplatform.BatchPredictionJob('projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248')
View Batch Prediction Job:
https://console.cloud.google.com/ai/platform/locations/europe-west1/batch-predictions/6547541440749109248?project=490069743207
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING


### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, you can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [15]:
batch_prediction_job.wait()

BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobState.JOB_STATE_RUNNING
BatchPredictionJob projects/490069743207/locations/europe-west1/batchPredictionJobs/6547541440749109248 current state:
JobSt

### Get the predictions and explanations

Next, get the results from the completed batch prediction job and print them out. Each result row will include the prediction and explanation.

In [16]:
for row in batch_prediction_job.iter_outputs():
    print(row)

Row(('96.0', '2023-03-30', '57.8', {'attributions': [{'featureAttributions': {'datetime': -0.19320297241210938, 'dew': 2.015615463256836, 'humidity': -9.615755081176758, 'precipcover': 0.0, 'windgust': 0.11479949951171875, 'windspeed': -0.24316978454589844, 'winddir': 0.14883804321289062, 'sealevelpressure': -0.12417793273925781, 'cloudcover': 3.8225479125976562, 'visibility': -1.35113525390625, 'solarradiation': -1.0856132507324219, 'solarenergy': 0.059993743896484375, 'uvindex': 2.6913795471191406, 'sunrise': -0.18847084045410156, 'sunset': 0.6783809661865234, 'tempmin': 0.0}, 'outputDisplayName': None}]}, '77.6', '1549', '0.0', '2023-03-30', {'value': 72.50701141357422}, {'value': 57.165863037109375}, '1014.1', '4.8', '54.6', '2023-03-30T07:17:55', '2023-03-30T19:45:54', None, None, '2', '9.5', '152.4', '36.3', '21.4'), {'cloudcover': 0, 'datetime': 1, 'dew': 2, 'explanation': 3, 'humidity': 4, 'idx': 5, 'precipcover': 6, 'predicted_on_datetime': 7, 'predicted_tempmax': 8, 'predicte

### Visualize the forecasts

Lastly, follow the given link to visualize the generated forecasts in [Data Studio](https://support.google.com/datastudio/answer/6283323?hl=en).
The code block included in this section dynamically generates a Data Studio link that specifies the template, the location of the forecasts, and the query to generate the chart. The data is populated from the forecasts generated using BigQuery options where the destination dataset is `batch_predict_bq_output_dataset_path`.

You can inspect the used template at https://datastudio.google.com/c/u/0/reporting/067f70d2-8cd6-4a4c-a099-292acd1053e8. This was created by Google specifically to view forecasting predictions.

**Note:** The Data Studio dashboard can only show the charts properly when the `batch_predict` job is run successfully using the BigQuery options.

In [39]:
time_column = "datetime"
time_series_identifier_column = "idx"
target_column = "tempmax"

In [40]:
tables = client.list_tables(batch_predict_bq_output_dataset_path)

prediction_table_id = ""
for table in tables:
    if (
        table.table_id.startswith("predictions_")
        and table.table_id > prediction_table_id
    ):
        prediction_table_id = table.table_id
batch_predict_bq_output_uri = "{}.{}".format(
    batch_predict_bq_output_dataset_path, prediction_table_id
)


def _sanitize_bq_uri(bq_uri):
    if bq_uri.startswith("bq://"):
        bq_uri = bq_uri[5:]
    return bq_uri.replace(":", ".")


def get_data_studio_link(
    batch_prediction_bq_input_uri,
    batch_prediction_bq_output_uri,
    time_column,
    time_series_identifier_column,
    target_column,
):
    batch_prediction_bq_input_uri = _sanitize_bq_uri(batch_prediction_bq_input_uri)
    batch_prediction_bq_output_uri = _sanitize_bq_uri(batch_prediction_bq_output_uri)
    base_url = "https://datastudio.google.com/c/u/0/reporting"
    query = (
        "SELECT \\n"
        " CAST(input.{} as DATETIME) timestamp_col,\\n"
        " CAST(input.{} as STRING) time_series_identifier_col,\\n"
        " CAST(input.{} as NUMERIC) historical_values,\\n"
        " CAST(predicted_{}.value as NUMERIC) predicted_values,\\n"
        " * \\n"
        "FROM `{}` input\\n"
        "LEFT JOIN `{}` output\\n"
        "ON\\n"
        "CAST(input.{} as DATETIME) = CAST(output.{} as DATETIME)\\n"
        "AND CAST(input.{} as STRING) = CAST(output.{} as STRING)"
    )
    query = query.format(
        time_column,
        time_series_identifier_column,
        target_column,
        target_column,
        batch_prediction_bq_input_uri,
        batch_prediction_bq_output_uri,
        time_column,
        time_column,
        time_series_identifier_column,
        time_series_identifier_column,
    )
    params = {
        "templateId": "067f70d2-8cd6-4a4c-a099-292acd1053e8",
        "ds0.connector": "BIG_QUERY",
        "ds0.projectId": PROJECT_ID,
        "ds0.billingProjectId": PROJECT_ID,
        "ds0.type": "CUSTOM_QUERY",
        "ds0.sql": query,
    }
    params_str_parts = []
    for k, v in params.items():
        params_str_parts.append('"{}":"{}"'.format(k, v))
    params_str = "".join(["{", ",".join(params_str_parts), "}"])
    return "{}?{}".format(base_url, urllib.parse.urlencode({"params": params_str}))


print(
    get_data_studio_link(
        PREDICTION_DATASET_BQ_PATH,
        batch_predict_bq_output_uri,
        time_column,
        time_series_identifier_column,
        target_column,
    )
)

https://datastudio.google.com/c/u/0/reporting?params=%7B%22templateId%22%3A%22067f70d2-8cd6-4a4c-a099-292acd1053e8%22%2C%22ds0.connector%22%3A%22BIG_QUERY%22%2C%22ds0.projectId%22%3A%22tidal-plasma-387718%22%2C%22ds0.billingProjectId%22%3A%22tidal-plasma-387718%22%2C%22ds0.type%22%3A%22CUSTOM_QUERY%22%2C%22ds0.sql%22%3A%22SELECT+%5Cn+CAST%28input.datetime+as+DATETIME%29+timestamp_col%2C%5Cn+CAST%28input.idx+as+STRING%29+time_series_identifier_col%2C%5Cn+CAST%28input.tempmax+as+NUMERIC%29+historical_values%2C%5Cn+CAST%28predicted_tempmax.value+as+NUMERIC%29+predicted_values%2C%5Cn+%2A+%5CnFROM+%60tidal-plasma-387718.hackathon_vertex_ai.xnew_automl_3_hor_sel_fea_Dallas%60+input%5CnLEFT+JOIN+%60tidal-plasma-387718.hackathon_vertex_ai_predictions.predictions_2023_07_09T13_27_27_268Z_825%60+output%5CnON%5CnCAST%28input.datetime+as+DATETIME%29+%3D+CAST%28output.datetime+as+DATETIME%29%5CnAND+CAST%28input.idx+as+STRING%29+%3D+CAST%28output.idx+as+STRING%29%22%7D


# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- AutoML Training Job
- Model
- Batch Prediction Job
- Cloud Storage Bucket

In [ ]:
# # Delete dataset
# dataset.delete()

# # Training job
# training_job.delete()

# # Delete model
# model.delete()

# Delete batch prediction job
batch_prediction_job.delete()

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI